In [ ]:
!rm -r /content/dats

rm: cannot remove '/content/dats': No such file or directory


In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git
!pip install torchxrayvision

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 3651 (delta 13), reused 21 (delta 8), pack-reused 3615
Receiving objects: 100% (3651/3651), 632.29 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (1455/1455), done.
Checking out files: 100% (1164/1164), done.
     |████████████████████████████████| 40.4MB 74kB/s 
     |████████████████████████████████| 35.5MB 90kB/s 


In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"randheerrameshk","key":"834af696027448aefdad59ac88e7f588"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:55<00:00, 78.8MB/s]
100% 2.29G/2.29G [00:55<00:00, 44.4MB/s]


In [ ]:
import os
import time
import copy
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import shutil
import math
from zipfile import ZipFile
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.optim import lr_scheduler

In [ ]:
shutil.copytree("/content/drive/My Drive/Dataset", "/content/data")

'/content/data'

In [ ]:
with ZipFile("/content/chest-xray-pneumonia.zip", 'r') as z :
  z.extractall()

In [ ]:
def create_dir_structure() :
  paths = [ "/content/Dataset/train/covid19",
            "/content/Dataset/train/normal",
            "/content/Dataset/train/pneumonia",
            "/content/Dataset/val/covid19",
            "/content/Dataset/val/normal",
            "/content/Dataset/val/pneumonia",
            "/content/covid19",
            "/content/normal",
            "/content/pneumonia" ]
  for path in paths :
    os.makedirs(path)
create_dir_structure()

In [ ]:
def covid19ImageFolder() :
  df = pd.read_csv("/content/covid-chestxray-dataset/metadata.csv")
  dataFrame = df[['finding', 'folder', 'filename']]
  i=0
  for finding, folder, filename in dataFrame.values :
    if(finding=="COVID-19") :
      try :
        img = Image.open("/content/covid-chestxray-dataset/"+folder+"/"+filename)
        i+=1
        shutil.copy("/content/covid-chestxray-dataset/"+folder+"/"+filename, "/content/covid19")
      except :
        print("cant open "+filename)

covid19ImageFolder()

In [ ]:
def normalImageFolder() :
  i=0
  for filename in os.listdir("/content/chest_xray/train/NORMAL") :
    if i<522:
      try :
        img = Image.open("/content/chest_xray/train/NORMAL/"+filename)
        i+=1
        shutil.copy("/content/chest_xray/train/NORMAL/"+filename, "/content/normal")
      except :
        print("cant open ")

normalImageFolder()

In [ ]:
def pneumoniaImageFolder() :   
  i=0
  for filename in os.listdir("/content/chest_xray/train/PNEUMONIA") :
    if i<522:
      try :
        img = Image.open("/content/chest_xray/train/PNEUMONIA/"+filename)
        i+=1
        shutil.copy("/content/chest_xray/train/PNEUMONIA/"+filename, "/content/pneumonia")
      except :
        print("cant open ")

pneumoniaImageFolder()

In [ ]:
def copyFiles(folderList) :
  for folder in folderList :
    i = 0
    for filename in os.listdir("/content/"+folder) :
      if(i<432) :
        shutil.copy("/content/"+folder+"/"+filename, "/content/Dataset/train/"+folder+"/")
      else :
        shutil.copy("/content/"+folder+"/"+filename, "/content/Dataset/val/"+folder+"/")
      i+=1
folderList = [ 'covid19', 'normal', 'pneumonia']
copyFiles(folderList)

In [ ]:
root = "/content/data"


In [ ]:
os.listdir(root)

['train', 'val']

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
image_datasets = {x: ImageFolder(os.path.join(root, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [ ]:
class_names = image_datasets['train'].classes

In [ ]:
class FCC(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        layer_sizes = [input_size] + hidden_layers + [output_size]
        self.classifier = nn.Sequential()
        for index, (inp, out) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            self.classifier.add_module('fc' + str(index), nn.Linear(inp, out))
            if index < (len(layer_sizes) - 2):  
                self.classifier.add_module('relu' + str(index + 1), nn.ReLU())
                self.classifier.add_module('dropout' + str(index + 1), nn.Dropout(drop_p))
        self.classifier.add_module('output', nn.LogSoftmax(dim=1))

In [ ]:
fcc = FCC(input_size=20588, output_size=102, hidden_layers=[2048, 512], drop_p=0.5)
print(fcc.classifier)

Sequential(
  (fc0): Linear(in_features=20588, out_features=2048, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=102, bias=True)
  (output): LogSoftmax(dim=1)
)


In [ ]:
def create_model(arch='vgg16', output_size=102, class_to_idx=None, hidden_layers=[4096, 4096], drop_p=0.5):
    input_size = 25088
    
    model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    
    fc = FCC(input_size=input_size, output_size=output_size,
                      hidden_layers=hidden_layers, drop_p=drop_p).classifier
    
    model.classifier = fc
    model.arch = arch
    model.class_to_idx = class_to_idx
    model.output_size = output_size
    model.hidden_layers = hidden_layers
    model.drop_p = drop_p
    
    return model

In [ ]:
model = create_model(arch='resnet18')
print(model)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
def create_optimizer(model, lr=0.001):
    params = model.classifier.parameters()
    optimizer = optim.Adam(params, lr=lr)
    return  optimizer

In [ ]:
optimizer = create_optimizer(model)
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [ ]:
def select_device(gpu):
    if not torch.cuda.is_available():
        print('GPU is not available')
        return torch.device('cpu')
    device = torch.device('cuda:0') if gpu else torch.device('cpu')  
    return device

In [ ]:
device = select_device(gpu=True)
print(device)

cuda:0


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, gpu=True,
                start_epoch=1, epochs=25):
    train_start = time.time()
    model.train()
    start_epoch=1
    best_acc = 0 
    best_epoch = 0
    best_model_state_dict = copy.deepcopy(model.state_dict())
    best_optimizer_state_dict = copy.deepcopy(optimizer.state_dict())
    device = select_device(gpu)
    model.to(device)
    for epoch in range(start_epoch, start_epoch + epochs):
        print(f'\nEpoch {epoch}/{start_epoch + epochs - 1}:'
              f'\n---------------------') 
        for phase in ['train', 'val']:
            phase_start = time.time()
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                
                if phase == 'train':
                    optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    pred = model(inputs)
                    loss = criterion(pred, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                predictions = torch.exp(pred)
                _, predictions = predictions.topk(1, dim=1)
                equals = predictions == labels.view(*predictions.shape)
                running_corrects += torch.sum(equals.type(torch.FloatTensor)).item()
            
            phase_loss = running_loss / len(dataloaders[phase].dataset)
            phase_acc = running_corrects / len(dataloaders[phase].dataset)
            
            if phase == 'val' and phase_acc > best_acc:
                best_epoch = epoch
                best_acc = phase_acc
                best_model_state_dict = copy.deepcopy(model.state_dict())
                best_optimizer_state_dict = copy.deepcopy(optimizer.state_dict())
                
            phase_duration = time.time() - phase_start
            print(f'{phase} Loss: {phase_loss:.4f}, Acc: {phase_acc:.4f}')
            
    model.load_state_dict(best_model_state_dict)
    optimizer.load_state_dict(best_optimizer_state_dict)
        
    train_duration = time.time() - train_start
    print(f'\nTraining complete in {(train_duration):.0f}s '
          f'Best Validation Acc: {best_acc:.4f}')
    
    return best_epoch

In [ ]:
model = create_model(arch='vgg16', output_size=102, class_to_idx=image_datasets['train'].class_to_idx,
                     hidden_layers=[2048, 512], drop_p=0.5)

optimizer = create_optimizer(model=model, lr=0.001)
criterion = nn.CrossEntropyLoss()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [ ]:
train_model(model=model, dataloaders=dataloaders, criterion=nn.CrossEntropyLoss(),
                                  optimizer=optimizer, gpu=True, start_epoch=1, epochs=25)


Epoch 1/25:
---------------------
train Loss: 1.2586, Acc: 0.6790
val Loss: 0.2645, Acc: 0.9061

Epoch 2/25:
---------------------
train Loss: 0.5362, Acc: 0.8032
val Loss: 0.2498, Acc: 0.9386

Epoch 3/25:
---------------------
train Loss: 0.5204, Acc: 0.8187
val Loss: 0.2550, Acc: 0.9242

Epoch 4/25:
---------------------
train Loss: 0.4960, Acc: 0.8264
val Loss: 0.2321, Acc: 0.9314

Epoch 5/25:
---------------------
train Loss: 0.5456, Acc: 0.8287
val Loss: 0.2056, Acc: 0.9422

Epoch 6/25:
---------------------
train Loss: 0.5130, Acc: 0.8202
val Loss: 0.5394, Acc: 0.7942

Epoch 7/25:
---------------------
train Loss: 0.5175, Acc: 0.8364
val Loss: 0.2063, Acc: 0.9495

Epoch 8/25:
---------------------
train Loss: 0.5565, Acc: 0.8164
val Loss: 0.2709, Acc: 0.9061

Epoch 9/25:
---------------------
train Loss: 0.4844, Acc: 0.8302
val Loss: 0.2278, Acc: 0.9314

Epoch 10/25:
---------------------
train Loss: 0.3607, Acc: 0.8673
val Loss: 0.1847, Acc: 0.9531

Epoch 11/25:
---------------

13

In [ ]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
def test(path) :
    test_transform = transforms.Compose([
                  transforms.Resize(256),
                  transforms.CenterCrop(224),
                  transforms.ToTensor(),
                  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
    img = Image.open(path).convert('RGB')
    img_tnsr = test_transform(img).float()
    img_tnsr = img_tnsr.unsqueeze_(0)
    print(f'predicted class: {class_names[model(img_tnsr.to(device)).data.cpu().numpy().argmax()]}')


In [ ]:
test('/content/data/test/covid19/covid19-10')

predicted class: covid19


In [ ]:
test('/content/data/test/covid19/covid19-6')

predicted class: covid19


In [ ]:
test('/content/data/test/normal/normal11')

predicted class: normal


In [ ]:
test('/content/data/test/normal/normal6')

predicted class: normal


In [ ]:
test('/content/data/test/pneumonia/pneumonia2')

predicted class: pneumonia


In [ ]:
test('/content/data/test/pneumonia/pneumonia8')

predicted class: pneumonia


In [ ]:
def save_checkpoint(model, optimizer):
    
    model.to('cpu')

    checkpoint = {
        'arch': model.arch,
        'output_size': model.output_size,
        'class_to_idx': model.class_to_idx,
        'hidden_layers': model.hidden_layers,
        'drop_p': model.drop_p,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    torch.save(checkpoint, "/content/data/checkpoint/checkpoint.0.1.pth")
    print('Checkpoint saved')

In [ ]:
save_checkpoint( model, optimizer)

Checkpoint saved
